In [1]:
import json
import pandas as pd
import sqlalchemy
import requests
from bs4 import BeautifulSoup


In [2]:
# Get keys from my_secrets_key.py file
from my_secrets_key import schema_name,host_name,user_name,db_password,port,weather_api_key,aerodatabox_api_key


In [3]:
schema = schema_name
host = host_name
user = user_name
password = db_password
port = port
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [4]:
# Function to extract city data from the soup object
def get_city(cities_soup):
  # Find the element with class 'infobox-label' and text 'Website'
  city_elem = cities_soup.find("span", {"class": "mw-page-title-main"})
  if city_elem:
    # If found, return the next sibling element with class 'infobox-data'
    city_element = city_elem.get_text(strip=True)
    return city_element
  else:
      # If not found, return None
      return None

# Function to extract country from the soup object
def get_country(cities_soup):
  country_elem = cities_soup.select("table.infobox td.infobox-data")[0].get_text(strip=True)
  if country_elem:
    country_element = country_elem
    return country_element
  else:
      # If not found, return None
      return "None"
    
# Function to extract coordinates from the soup object
def get_coordinates(cities_soup, coord_type):
  coord_elem = cities_soup.select("span."+coord_type)[0].get_text(strip=True)
  return coord_elem

# Function to extract population from the soup object
def get_population(cities_soup):
    # Find the table header element containing the text 'Population'
    population_elem = cities_soup.select_one('th.infobox-header:-soup-contains("Population")') or cities_soup.select_one('th.infobox-label:-soup-contains("Population")')
    # If found, find the next sibling and extract the first numerical data
    if population_elem:
      population = population_elem.parent.find_next('td', class_='infobox-data').get_text(strip=True)
      return population
    else:
      None

def clean_data(df):
    # Remove unnecessary characters from 'longitude' 'latitude' and 'population' columns
    df['longitude'] = df['longitude'].str.replace('°', '.').str.replace('′', '').str.replace('″', '')
    df['latitude'] = df['latitude'].str.replace('°', '.').str.replace('′', '').str.replace('″', '')
    df['population'] = df['population'].str.replace(r'[\[\],]', '', regex=True)


 
# Define a function to extract city information
def extract_city_info(cityname):
    # Send an HTTP GET request to the Wikipedia page
    city_url = f'https://en.wikipedia.org/wiki/{cityname}'
    response = requests.get(city_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page using BeautifulSoup
        cities_soup = BeautifulSoup(response.text, "html.parser")

        # Initialize variables to store information
        country = None  # Since we are focusing on  cities
        latitude = None
        longitude = None
        population = None
        city    = None
        
        city = get_city(cities_soup)
        country_name = get_country(cities_soup) 
        latitude = get_coordinates(cities_soup,"latitude") 
        longitude = get_coordinates(cities_soup,"longitude")
        population = get_population(cities_soup)
        city_data = {
              "city": city,
              "country": country_name,
              "latitude": latitude,
              "longitude": longitude,
              "population": population
       }
        return city_data
        
    else:
        print(f"Failed to retrieve data for {cityname}")


def city_info(list_of_cities):
    
    city_data = [extract_city_info(city) for city in list_of_cities]
    # Convert the list of dictionaries to a DataFrame
    cities_df = pd.DataFrame(city_data)
    # Clean the data
    clean_data(cities_df)
    # Return the DataFrame
    return cities_df
    

# Iterate through the list of  cities and extract information for each
list_of_cities = ['Berlin', 'Hamburg', 'Munich','Paris', 'London','Stuttgart','Tokyo','Trichy']
cities_data_df = city_info(list_of_cities)



Created Dataframe population and cities

In [5]:
gans_cities_df = pd.DataFrame({
    'city_name': cities_data_df['city'], 
    'country': cities_data_df['country']
})

gans_populations_df = pd.DataFrame({
    'city_name': cities_data_df['city'], 
    'latitude': cities_data_df['latitude'],
    'longitude': cities_data_df['longitude'],
    'population': cities_data_df['population']
})

Stores city data into cities table

In [6]:
gans_cities_df.to_sql(name='cities', con=con, if_exists='append', index=False)


8

In [7]:

cities_res = pd.read_sql_table("cities", con=con,columns=['city_id','city_name'])
cities_res

,city_id,city_name
0,25,Berlin
1,26,Hamburg
2,27,Munich
3,28,Paris
4,29,London
5,30,Stuttgart
6,31,Tokyo
7,32,Tiruchirappalli


In [10]:
# Merge the two DataFrames using 'city_name' as the key
gans_populations_data = gans_populations_df.merge(cities_res, on='city_name')
# Drop the redundant 'city'  column if needed
gans_populations_data = gans_populations_data.drop(columns=['city_name'])

gans_populations_data

,latitude,longitude,population,city_id
0,52.3112N,13.2418E,3850809,25
1,53.33N,10.00E,1945532,26
2,48.0815N,11.3430E,1512491,27
3,48.5124N,2.218E,2102650,28
4,51.3026N,0.739W,87998001,29
5,48.4639N,09.1048E,626275,30
6,35.4123N,139.4132E,14094034,31
7,10.4725N,78.4217E,9168571,32


In [11]:
gans_populations_data.to_sql(name='populations', con=con, if_exists='append', index=False)


8